In [1]:
import numpy as np
import pandas as pd
import datetime as dt

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import seaborn as sns
#import mplcursors

import sys
import operator

import fastparquet
import snappy

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

/Users/edasmalchi/anaconda3/envs/pd1/lib/python3.8/site-packages/fastparquet/dataframe.py:5: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import CategoricalIndex, RangeIndex, Index, MultiIndex


In [2]:
 def showall(df):
    #shows entire dataframe
    assert df.shape[0] < 5000
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        display(df)

In [3]:
alldays_timestamped = pd.read_parquet('TimestampToSep11.parquet')
alldays_timestamped.sort_index(level='Timestamp', inplace=True)

In [9]:
def make_plot(df, palette, save):
    #return int for size based on plot duration
    def dynamic(duration_min):
        if duration_min < 2:
            return 7
        elif duration_min < 5:
            return 6
        elif duration_min < 15:
            return 5
        elif duration_min < 60:
            return 3
        else:
            return 2
    #suppress Pandas view vs copy warning (seems to work ok here)
    with pd.option_context('mode.chained_assignment', None):
        #remove locations besides spc 1-3, bike lane
        filtered_df = df[~df['Vehicle Location'].isin(['SB travel lane', 
                                                        'NB right turn pocket', 'Both/Middle of Space 1 & 2'])]
        #order locations
        filtered_df.sort_values(by=['Vehicle Location'], inplace=True)
        #create time column from index for x-axis, assign enforcement start/end times
        filtered_df['Time'] = filtered_df.index
        start = min(filtered_df.index).to_pydatetime()
        end = max(filtered_df.index).to_pydatetime()
        duration = ((end - start).seconds) / 60
        enf_start = start.replace(hour=18, minute=0)
        enf_end = start.replace(hour=22, minute=0)
        
        fig, ax = plt.subplots()
        #plot using Seaborn strip plot, set x-axis range, add line at enforcement start/end time
        #hue based on 'Violator' values, which include TNC/CNS status 
        ax = sns.stripplot(x="Time", y="Vehicle Location", hue='Violator', palette=palette, data=filtered_df,
                           size = dynamic(duration), jitter=False)
        ax.set_xlim([start, end])
        ax.axvline(enf_start, label='Loading Zone Start', c='r')
        ax.axvline(enf_end, label='Loading Zone End', c='b')
        #move legend
        ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
        #title plot with date
        ax.set_title(filtered_df['Begin Date'][0])
        #format times on x-axis to readable '9:30PM' format
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%-I:%M %p'))
        fig.autofmt_xdate()
        #Save to figures folder with start/end times in filename
        if save:
            plt.savefig('Figures/{}–{}.png'.format(start, end), dpi=600, format="png", bbox_inches='tight')
        plt.show()
        return ax

In [6]:
def plot(df, allday_df=alldays_timestamped, save=False):
    sns.set_style('white')
    #get/sort unique Violator types
    unique = allday_df['Violator'].unique()
    unique.sort()
    #colors from xkcd colors https://xkcd.com/color/rgb/ 
    colors = ['black', 'scarlet', 'light red', 
              'olive green', 'grass green', 'mint', 
              'brick red', 'dark orange', 'pumpkin']
    #zip Violator types to color values
    palette = dict(zip(unique, sns.xkcd_palette(colors)))
    #call to make actual plot
    try:
        fig = make_plot(df, palette, save)
    except:
        print('Please select a valid time range.')
    return

In [33]:
dt1 = dt.datetime.combine(dt.date.today(), dt.datetime.strptime('06:00PM', '%I:%M%p').time())
dt2 = dt.datetime.combine(dt.date.today(), dt.time.fromisoformat('22:00:00'))

In [39]:
(dt2 - dt1).seconds / 60

240.0

In [30]:
#dt.datetime.strptime('06:00PM', '%I:%M%p')
dt.datetime.fromisoformat('22:00:00')

ValueError: Invalid isoformat string: '22:00:00'

In [47]:
##evidently broke something

time_widget = widgets.SelectionRangeSlider(options=[0], continuous_update=False, layout={'width': '400px'})
date_widget = widgets.Dropdown(options=alldays_timestamped['Begin Date'].unique())

def update(*args):
    index = alldays_timestamped[alldays_timestamped['Begin Date'] == date_widget.value].index
    minutes = index.strftime('%I:%M%p').unique()
    time_widget.options = minutes
date_widget.observe(update)

def iplot(Date, Time, df, enf_start, enf_end):
    datefiltered = alldays_timestamped[alldays_timestamped['Begin Date'] == Date]
    enf_dur = (dt.datetime.combine(dt.date.today(),dt.time.fromisoformat(enf_end)) 
                   - dt.datetime.combine(dt.date.today(),dt.time.fromisoformat(enf_start)))
    try:
        start_dt = dt.datetime.strptime(Time[0], '%I:%M%p')
        start_time = start_dt.time()
        end_dt = dt.datetime.strptime(Time[1], '%I:%M%p')
        end_time = end_dt.time()
        
        timefiltered = datefiltered[datefiltered.index.time > start_time]
        timefiltered = timefiltered[timefiltered.index.time < end_time]
        #debug st
        if start_time < dt.time.fromisoformat(enf_start) and end_time < dt.time.fromisoformat(enf_end):
            keep = 'last'
        elif start_time > dt.time.fromisoformat(enf_start) and end_time > dt.time.fromisoformat(enf_end):
            keep = 'first'
        else:
            keep = 'last'
            longest_dur = timefiltered['Duration'].max()
            to_td = dt.datetime.strptime(longest_dur, '%-H:%M:%S')
            td = dt.timedelta(hours=to_td.hour, minutes=to_td.minute, seconds=to_td.second)
            if td > enf_dur:
                print('Violator classification may be inaccurate for vehicles spanning entire enforcement duration\n',
                      'Coloring on chart is accurate')
        #debug end
        plot(timefiltered)
        with pd.option_context('mode.chained_assignment', None):
            #Drop space 3 and minor locations to focus on Space 1/Space 2
            top_durations = timefiltered[~timefiltered['Vehicle Location'].isin(['Space 3', 'SB travel lane', 
                                                                'NB right turn pocket', 'Both/Middle of Space 1 & 2'])]
            #Drop less useful columns to present clear summary
            top_durations.drop(['Begin Date', 'Vehicle Type', 'Vehicle Characteristics', 
                                    'Bikeway Users Displaced', 'LZ Space Avail', 
                                    'Occupied while idle?', 'CNS?', 'TNC?'], axis=1, inplace=True)
            top_durations = top_durations.drop_duplicates(['Duration'])
            top_durations = top_durations.sort_values(by='Duration', ascending=False)
            top_durations = top_durations[:10].set_index('Vehicle Location')
            display(top_durations)
    except:
        print('Please select a valid time range.')

In [43]:
alldays_timestamped.loc['2019-09-03 17:59:58':'2019-09-03 18:00:02']['Duration'].max()

'5:30:00'

In [46]:
alldays_timestamped.loc['2019-09-03 17:59:58':'2019-09-03 19:00:02']['Duration'].min()

'0:00:38'

In [48]:
interact(iplot, Date=date_widget, Time=time_widget, 
         df=fixed(alldays_timestamped), enf_start=fixed('18:00:00'),
        enf_end=fixed('22:00:00'));

interactive(children=(Dropdown(description='Date', options=('08/21/2019', '08/22/2019', '08/23/2019', '08/24/2…

TraitError: Invalid selection: index out of bounds: (0, 626)

## Moving Forwards
* buttons to save plot, generate+download stats... 
* fix 9/5 8:07pm on Google?